In [66]:
# https://kanoki.org/2019/02/26/compare-two-excel-files-for-difference-using-python/

import pandas as pd

In [67]:
oms_df = pd.read_csv("C:/PythonProjects/data/20221214_trade.csv", encoding='euc-kr')
trader_df = pd.read_excel("C:/PythonProjects/data/일일매매내역양식.xlsx", )

In [68]:
oms_df

,단축코드,종목명,매매구분,실주문수량,단가,체결수량,체결금액,체결단가,체결잔량,현재가,등락,등락율,수익율,매매유형코드,매매유형,매매구분명,STP체결율,FIX체결율,종목코드,집행대비등락률
0,100,유한양행,1,845,59700,845,50394800,59639,0,59700,0,0.000000,-0.102485,1,일반,매도,100,100,KR7000100008,0.000000
1,5380,현대차,2,2195,159500,2195,351200000,160000,0,163500,4000,2.507837,2.187500,1,일반,매수,100,100,KR7005380001,2.507837
2,34220,LG디스플레이,2,7333,13900,7333,102845550,14025,0,14250,600,4.395604,1.604056,4,차입,매수,100,100,KR7034220004,2.689865
3,96530,씨젠,2,4500,28000,4500,128059650,28458,0,28250,250,0.892857,-0.729855,4,차입,매수,100,100,KR7096530001,0.892857
4,137310,에스디바이오센서,1,6100,33850,6100,198614500,32560,0,32100,-350,-1.078582,1.432256,4,차입,매도,100,100,KR7137310009,-5.233829
5,141080,레고켐바이오,1,1400,38200,1400,54878650,39199,0,39150,950,2.486911,0.125251,1,일반,매도,100,100,KR7141080002,2.486911
6,226490,KODEX 코스피,1,4710,24350,4710,114713650,24355,0,24405,245,1.014073,-0.203484,1,일반,매도,100,100,KR7226490001,0.401474
7,229200,KODEX 코스닥150,1,13550,10345,13550,140493165,10368,0,10390,230,2.263780,-0.206937,1,일반,매도,100,100,KR7229200001,1.115250
8,252670,KODEX 200선물인버스2X,2,17850,3085,17850,55053170,3084,0,3075,-65,-2.070064,-0.298657,1,일반,매수,100,100,KR7252670005,-0.435944
9,302440,SK바이오사이언스,2,1375,83000,1375,118145900,85924,0,89200,6200,7.469880,3.812320,4,차입,매수,100,100,KR7302440003,7.469880


In [69]:
trader_df

,펀드,매매처,단축코드,종목명,매매구분,체결수량,체결단가,체결금액
0,하이일드,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,공모주1호,Yuanta,34220.0,LG디스플레이,Buy cover,7333.0,14025.0,102845325.0
7,NaN,NH,137310.0,에스디바이오센서,Short sell,6100.0,32560.0,-198616000.0
8,NaN,Yuanta,5380.0,현대차,Buy,630.0,160000.0,-100800000.0
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [70]:
# https://sparkbyexamples.com/pandas/pandas-select-columns-by-name-or-index/
# 필요한 열 선택

oms_df1 = oms_df[["종목명", "매매유형", "매매구분명", "체결수량", "체결단가", "체결금액"]]

In [71]:
oms_df1

,종목명,매매유형,매매구분명,체결수량,체결단가,체결금액
0,유한양행,일반,매도,845,59639,50394800
1,현대차,일반,매수,2195,160000,351200000
2,LG디스플레이,차입,매수,7333,14025,102845550
3,씨젠,차입,매수,4500,28458,128059650
4,에스디바이오센서,차입,매도,6100,32560,198614500
5,레고켐바이오,일반,매도,1400,39199,54878650
6,KODEX 코스피,일반,매도,4710,24355,114713650
7,KODEX 코스닥150,일반,매도,13550,10368,140493165
8,KODEX 200선물인버스2X,일반,매수,17850,3084,55053170
9,SK바이오사이언스,차입,매수,1375,85924,118145900


In [72]:
# Short sell, Buy cover, Buy, Sell 매매구분을 위한 열 병합 및 값 변환
oms_df1["매매구분"] = oms_df1[["매매유형", "매매구분명"]].agg('_'.join, axis=1)

oms_df1["매매구분"] = oms_df1["매매구분"].replace(['일반_매수', '일반_매도', '차입_매수', '차입_매도'], ['Buy', 'Sell', 'Buy cover', 'Short sell'])

oms_df1 = oms_df1.drop(oms_df1.columns[[1,2]], axis=1)

col_to_move = oms_df1.pop('매매구분')

oms_df1.insert(1, '매매구분', col_to_move)

oms_df1

C:\Users\jtlee\AppData\Local\Temp\ipykernel_27516\3116984185.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oms_df1["매매구분"] = oms_df1[["매매유형", "매매구분명"]].agg('_'.join, axis=1)
C:\Users\jtlee\AppData\Local\Temp\ipykernel_27516\3116984185.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oms_df1["매매구분"] = oms_df1["매매구분"].replace(['일반_매수', '일반_매도', '차입_매수', '차입_매도'], ['Buy', 'Sell', 'Buy cover', 'Short sell'])


,종목명,매매구분,체결수량,체결단가,체결금액
0,유한양행,Sell,845,59639,50394800
1,현대차,Buy,2195,160000,351200000
2,LG디스플레이,Buy cover,7333,14025,102845550
3,씨젠,Buy cover,4500,28458,128059650
4,에스디바이오센서,Short sell,6100,32560,198614500
5,레고켐바이오,Sell,1400,39199,54878650
6,KODEX 코스피,Sell,4710,24355,114713650
7,KODEX 코스닥150,Sell,13550,10368,140493165
8,KODEX 200선물인버스2X,Buy,17850,3084,55053170
9,SK바이오사이언스,Buy cover,1375,85924,118145900


#둘 다 키값을 종목명(Key1) and 체결수량(Key2)으로 잡고 정렬한 다음 데이터프레임의 밸류들이 일치하는지 확인

In [73]:
oms_df2 = oms_df1.set_index('종목명')

oms_df3 = oms_df2.sort_index(axis=0)

In [74]:
oms_df3

,매매구분,체결수량,체결단가,체결금액
종목명,,,,
KODEX 200선물인버스2X,Buy,17850,3084,55053170
KODEX 코스닥150,Sell,13550,10368,140493165
KODEX 코스피,Sell,4710,24355,114713650
LG디스플레이,Buy cover,7333,14025,102845550
SK바이오사이언스,Buy cover,1375,85924,118145900
레고켐바이오,Sell,1400,39199,54878650
박셀바이오,Buy cover,3040,39497,120070800
씨젠,Buy cover,4500,28458,128059650
에스디바이오센서,Short sell,6100,32560,198614500


In [75]:
trader_df1 = trader_df[["종목명", "매매구분", "체결수량", "체결단가", "체결금액"]]

In [76]:
trader_df1 = trader_df1.dropna()

trader_df2 = trader_df1.set_index('종목명')

col_to_move2 = trader_df2.pop('매매구분')

col_to_move2 = col_to_move2.reset_index()

col_to_move3 = col_to_move2.drop_duplicates(subset=['종목명'], keep='first')

col_to_move3 = col_to_move3.set_index('종목명')

trader_df2 = trader_df1.groupby(trader_df1['종목명'], sort=False).sum()

# trader_df2 = trader_df2.reset_index()
 
trader_df3 = trader_df2.merge(col_to_move3, how='outer', left_index=True, right_index=True)

trader_df3 = trader_df3[['매매구분', '체결수량', '체결단가', '체결금액']]

# trader_df2 = trader_df1.set_index('종목명')

# trader_df3 = trader_df2.sort_index(axis=0)

C:\Users\jtlee\AppData\Local\Temp\ipykernel_27516\1764132459.py:13: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  trader_df2 = trader_df1.groupby(trader_df1['종목명'], sort=False).sum()


In [77]:
# col_to_move2 = col_to_move2.reset_index()
col_to_move3 = col_to_move2.drop_duplicates(subset=['종목명'], keep='first')

In [78]:
col_to_move3

,종목명,매매구분
0,LG디스플레이,Buy cover
1,에스디바이오센서,Short sell
2,현대차,Buy
4,유한양행,Sell
5,레고켐바이오,Sell
6,씨젠,Buy cover
7,SK바이오사이언스,Buy cover
8,박셀바이오,Buy cover
9,KODEX 코스닥150,Sell
10,KODEX 코스피,Sell


In [84]:
trader_df3 = trader_df3.sort_index(axis=0)

In [87]:
trader_df3 = trader_df3.astype({'체결수량' : 'int', '체결단가' : 'int', '체결금액' : 'int'})

In [89]:
trader_df3

,매매구분,체결수량,체결단가,체결금액
종목명,,,,
KODEX 200인버스2x,Buy,17850,6169,-55050900
KODEX 코스닥150,Sell,13550,10368,140486400
KODEX 코스피,Sell,4710,24355,114712050
LG디스플레이,Buy cover,7333,14025,102845325
SK바이오사이언스,Buy cover,1375,171645,118145850
레고켐바이오,Sell,1400,39199,54878600
박셀바이오,Buy cover,3040,79003,120070000
씨젠,Buy cover,4500,28458,128061000
에스디바이오센서,Short sell,6100,32560,-198616000


In [90]:
oms_df3.compare(trader_df3)

ValueError: Can only compare identically-labeled DataFrame objects

In [80]:
# trader_df4

# col_to_move2 = trader_df3.pop('매매구분')

# col_to_move2.reset_index()

# trader_df4 = trader_df3.groupby(trader_df3.index, sort=False).sum()

# trader_df4.insert(1, '매매구분', col_to_move2)

# trader_df4